## AM 216 Final Project
Team Member: Lihong Zhang, Litao Yan, Ruoxi Yang

Deep learning bears promise for drug discovery, and recently we have seen many prospective application in the field of drug-target interaction. However, the underlying mathematical models often remain elusive to interpretation, since ofen time deep pearning can only be treated as black box and hard to interpret. 
For this final project, inspired by Drug-target interaction miniproject from this course, we want to analyze explainabliity of different data representations. 

We explore the representations and their visualization from different perspective. Due to the time limit, we write the final project in 2 parts.

## 1 Graph Neural Networks
Inspired by section 10, we intend to apply Graph Neural Networks(GNN) in solving drug-protein binding affinity problem and see whether GNN would improve the prediction results.

In [27]:
import numpy as np
import rdkit
from rdkit.Chem import Draw
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import MolFromSmiles

import tensorflow as tf
from keras import Sequential, Model, Input
from keras.layers import Dense, concatenate, Dropout
from tensorflow.keras.models import Model
from keras import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Conv2D, MaxPooling2D

from spektral.layers import GATConv, ChebConv

import pandas as pd
import matplotlib.pyplot as plt
import json
import pickle
from collections import OrderedDict
import networkx as nx

from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol

import sys
# sys.path.append('/usr/local/lib/python3.7/site-packages/')
import deepchem as dc

Prepare train data and test data

In [2]:
LOCAL_KIBA_PATH = '../data/mini_project_data/data/kiba/'
LOCAL_DAVIS_PATH = '../data/GraphDTA_davis/'
G_PATH = './drive/MyDrive/Colab Notebooks/Drug Binding'
max_seq_len = 1000
# for converting protein sequence to categorical format/numerical format
seq_voc = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
seq_dict = {v:i for i,v in enumerate(seq_voc)}
seq_dict_len = len(seq_dict)


In [10]:
# !! 13 2x2 matrices, nodes + adj together

def adj2matr(raw_a):
    len_a = len(raw_a)
    matr = np.zeros((len_a, len_a))
    for i in range(len_a):
        for j in raw_a[i]:
            matr[i, j] = 1
    return matr

# convmol to matrices
def conv2matr(convmol):
    nr, nc = max([convmol[i].get_atom_features().shape[0] for i in range(len(convmol))]), convmol[0].get_atom_features().shape[1]
    amax = max([len(convmol[i].get_adjacency_list()) for i in range(len(convmol))])
    ar, ac = amax, amax
    nodes = []
    adj = []
    for i in range(len(convmol)):
        temp_n = np.zeros((nr, nc))
        n = convmol[i].get_atom_features()
        temp_n[:n.shape[0],:n.shape[1]] = n
        # nodes.append(temp_n)
        nodes.append(temp_n.reshape((1, temp_n.shape[0], temp_n.shape[1])))
        temp_a = np.zeros((ar, ac))
        a = adj2matr(convmol[i].get_adjacency_list()) 
        temp_a[:a.shape[0],:a.shape[1]] = a
        # adj.append(temp_a)
        adj.append(temp_a.reshape((1, temp_a.shape[0], temp_a.shape[1])))
    return [np.concatenate(nodes, axis = 0), np.concatenate(adj, axis = 0)]
    # return [nodes, adj]



# The codes below in this cell are from section 10 of AM216 and mini project
def seq_to_cat(prot):  # prot: protein
    x = np.zeros(max_seq_len)
    for i, ch in enumerate(prot[:max_seq_len]): 
        x[i] = seq_dict[ch]
    return x  

def read_data(data_path):
    drugs_ = json.load(open(data_path + "ligands_can.txt"), object_pairs_hook=OrderedDict)
    
    smiles = np.array([Chem.MolToSmiles(Chem.MolFromSmiles(d),isomericSmiles=True) for d in drugs_.values()])
    featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)
    convmol = featurizer.featurize(smiles)

    # inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    # for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
      # inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
      
    # drugs = conv2matr(convmol)

    proteins_ = json.load(open(data_path + "proteins.txt"), object_pairs_hook=OrderedDict)
    proteins = np.array(list(proteins_.values()))
    affinity = np.array(pickle.load(open(data_path + "Y","rb"), encoding='latin1'))
    train_fold = json.load(open(data_path + "folds/train_fold_setting1.txt"))
    train_fold = [ee for e in train_fold for ee in e ]    
    test_fold = json.load(open(data_path + "folds/test_fold_setting1.txt"))

    # Prepare train/test data with fold indices
    rows, cols = np.where(np.isnan(affinity)==False) 
    convmol_tr = convmol[rows[train_fold]]    # (98545,)
    smiles_tr = smiles[rows[train_fold]] 
    proteins_tr = np.array([seq_to_cat(p) for p in proteins[cols[train_fold]]])   # (98545, 1000)
    affinity_tr = affinity[rows[train_fold], cols[train_fold]]  # (98545,)

    convmol_ts = convmol[rows[test_fold]] # (19709,)
    smiles_ts = smiles[rows[test_fold]] # (19709,)
    proteins_ts = np.array([seq_to_cat(p) for p in proteins[cols[test_fold]]]) # (19709, 1000)
    affinity_ts = affinity[rows[test_fold], cols[test_fold]]    # (19709,)
    '''
    print('Example of drug:{}'.format(drugs_tr[0]))
    print('Example of protein:{} ...'.format(proteins_tr[0][:10]))
    print('Example of affinity score:{}'.format(affinity_tr[0]))
    '''
    return convmol_tr, smiles_tr, proteins_tr, affinity_tr, convmol_ts, smiles_ts, proteins_ts, affinity_ts

def smiles_graph(path):
    drugs_ = json.load(open(path + 'ligands_can.txt'), object_pairs_hook=OrderedDict)
    # print('\nOriginal molecule:')
    mols = MolFromSmiles(smiles[0])
    # Draw.MolToImage(mols)
    featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)
    graph_data = featurizer.featurize(smiles)
    return smiles, graph_data

In [35]:
nr, nc = max([convmol_tr_kiba[i].get_atom_features().shape[0] for i in range(len(convmol_tr_kiba))]), convmol_tr_kiba[0].get_atom_features().shape[1]

Convert SMILES to Graphs

In [4]:
convmol_tr_kiba, smiles_tr_kiba, proteins_tr_kiba, affinity_tr_kiba, convmol_ts_kiba, smiles_ts_kiba, proteins_ts_kiba, affinity_ts_kiba = read_data(LOCAL_KIBA_PATH)
convmol_tr_davis, smiles_tr_davis, proteins_tr_davis, affinity_tr_davis, convmol_ts_davis, smiles_ts_davis, proteins_ts_davis, affinity_ts_davis = read_data(LOCAL_DAVIS_PATH)

25046

In [11]:
# get the nodes and adjacency matrix of drugs
# nodes_tr_kiba, adj_tr_kiba = conv2matr(convmol_tr_kiba)
# nodes_ts_kiba, adj_ts_kiba = conv2matr(convmol_ts_kiba)

nodes_tr_davis, adj_tr_davis = conv2matr(convmol_tr_davis)
nodes_ts_davis, adj_ts_davis = conv2matr(convmol_ts_davis)

In [18]:
# print(nodes_ts_kiba.shape, nodes_ts_kiba[0].shape, adj_ts_kiba.shape)
print(nodes_tr_davis.shape, adj_tr_davis.shape)

(25046, 46, 75) (25046, 46, 46)


Prepare data for CNN

In [15]:
# tr_size, drug_size = drugs_ecfp_tr.shape[0], drugs_ecfp_tr.shape[1]
num_ts_davis = proteins_ts_davis.shape[0]

# num_train, num_drugs = drugs_ecfp_tr.shape
num_tr_davis, num_prot_davis = proteins_tr_davis.shape
# drugs_tr_reshape = drugs_ecfp_tr.reshape((num_train, num_drugs, 1))
proteins_tr_davis_reshape = proteins_tr_davis.reshape((num_tr_davis, num_prot_davis, 1))

# Testing data
# drug_ts_reshape = drugs_ecfp_ts.reshape((drugs_ecfp_ts.shape[0], drugs_ecfp_ts.shape[1], 1))
# proteins_ts_reshape = proteins_ts.reshape((proteins_ts.shape[0], proteins_ts.shape[1], 1))

In [16]:
print(affinity_tr_davis.shape, proteins_tr_davis_reshape.shape)

(25046,) (25046, 1000, 1)


Prepare data for GNN

In [ ]:
train_dataset = dc.data.NumpyDataset([convmol_tr_davis, proteins_tr_davis_reshape], affinity_tr_davis)

In [33]:
# https://stackoverflow.com/questions/62232435/error-in-multi-input-model-for-graph-neural-network-in-tf-keras

# CNN for protein
Feat_input = Input(shape=(num_prot_davis,1))
Feat_layer = Conv1D(16, 3, activation='relu', input_shape=(num_prot_davis, 1))(Feat_input)
Feat_layer = MaxPooling1D(3)(Feat_layer)
Feat_layer = Flatten()(Feat_layer)
Feat_layer = Dropout(0.1)(Feat_layer)
Feat_layer = Dense(16, activation = 'linear')(Feat_layer)

xs1, xs2 = nodes_ts_davis.shape[1], nodes_ts_davis.shape[2]
as1, as2 = adj_tr_davis.shape[1], adj_tr_davis.shape[2]
X_in = Input(shape=(xs1, xs2))
A_in = Input(shape = (as1, as2))

# GNN for 
graph_conv = GATConv(64, activation='relu',name='graph_input')([X_in, A_in])
graph_conv = Dropout(0.5)(graph_conv)

graph_conv = ChebConv(32, activation='relu')([graph_conv,A_in])

graph_conv = Dropout(0.5)(graph_conv)

graph_conv = GATConv(64, activation='relu')([graph_conv,A_in])
graph_conv = Dropout(0.5)(graph_conv)
graph_conv = ChebConv(32, activation='relu')([graph_conv, A_in])

graph_conv = Flatten()(graph_conv)

dta_concat = concatenate([graph_conv, Feat_layer])

dta_concat = Dense(512, activation='relu')(dta_concat)
dta_concat = Dense(256, activation='relu')(dta_concat)
dta_concat = Dense(1, activation='softmax')(dta_concat)

model = Model(inputs={'graph_input':[X_in, A_in], 'cnn_input':Feat_input}, outputs=dta_concat)



model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 46, 75)]     0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, 46, 46)]     0                                            
__________________________________________________________________________________________________
graph_input (GATConv)           (None, 46, 64)       4992        input_18[0][0]                   
                                                                 input_19[0][0]                   
__________________________________________________________________________________________________
dropout_15 (Dropout)            (None, 46, 64)       0           graph_input[0][0]          

In [32]:
history = model.fit({'graph_input': [nodes_tr_davis, adj_tr_davis], 'cnn_input': proteins_tr_davis_reshape }, affinity_tr_davis,batch_size=28, epochs=250,steps_per_epoch=10)

5455 - acc: 0.0000e+00
Epoch 65/250
10/10 [==============================] - 0s 18ms/step - loss: 72060014.5455 - acc: 0.0000e+00
Epoch 66/250
10/10 [==============================] - 0s 17ms/step - loss: 65278169.0909 - acc: 0.0000e+00
Epoch 67/250
10/10 [==============================] - 0s 15ms/step - loss: 70486602.1818 - acc: 0.0000e+00
Epoch 68/250
10/10 [==============================] - 0s 17ms/step - loss: 68107737.8182 - acc: 0.0000e+00
Epoch 69/250
10/10 [==============================] - 0s 17ms/step - loss: 67558012.0000 - acc: 0.0000e+00
Epoch 70/250
10/10 [==============================] - 0s 15ms/step - loss: 70086497.4545 - acc: 0.0000e+00
Epoch 71/250
10/10 [==============================] - 0s 18ms/step - loss: 69853175.2727 - acc: 0.0000e+00
Epoch 72/250
10/10 [==============================] - 0s 17ms/step - loss: 74343170.9091 - acc: 0.0000e+00
Epoch 73/250
10/10 [==============================] - 0s 18ms/step - loss: 72277384.0000 - acc: 0.0000e+00
Epoch 74/250
1

Build a 2D CNN based on graph DAVIS data




In [9]:


nodes = nodes_ts_davis[0]
adj = adj_ts_davis[0]

def cnn1d(input_dim):
    cnn = Sequential() # Create sequential model
    cnn.add(Conv1D(16, 3, activation='relu', input_shape=(input_dim, 1)))
    cnn.add(MaxPooling1D(3))
    cnn.add(Flatten())
    cnn.add(Dropout(0.1))
    cnn.add(Dense(16, activation = 'linear'))
    return cnn

def cnn2d_nodes(nodes):
    cnn = Sequential() # Create sequential model
    cnn.add(Conv2D(16, 3, activation='relu', input_shape=nodes.shape))
    cnn.add(MaxPooling2D(3))
    cnn.add(Flatten())
    cnn.add(Dropout(0.1))
    cnn.add(Dense(16, activation = 'linear'))
    return cnn

def cnn2d_adj(adj):
    cnn = Sequential() # Create sequential model
    cnn.add(Conv2D(16, 3, activation='relu', input_shape=adj.shape))
    cnn.add(MaxPooling2D(3))
    cnn.add(Flatten())
    cnn.add(Dropout(0.1))
    cnn.add(Dense(16, activation = 'linear'))
    return cnn

nodes_cnn = cnn2d_nodes(nodes)
adj_cnn = cnn2d_adj(adj)
prot_cnn = cnn1d(input_dim)
cnn_concat = concatenate([nodes_cnn.output, adj_cnn.output, prot_cnn.output])

final_concat = Model(inputs=[[nodes_cnn.input, adj_cnn.input], prot_cnn.input], outputs=cnn_concat)


final_concat = Dense(1024, activation='relu')(final_concat)
final_concat = Dropout(0.1)(cnn_concat)
final_concat = Dense(16, activation='relu')(final_concat)

final = Dense(1, activation='linear')(final_concat)


# Show model summary
final.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
final.summary()

NameError: name 'num_drugs' is not defined

In [ ]:
# Convert SMILES to graphs
smiles_kiba, convmol_kiba = smiles_graph(LOCAL_KIBA_PATH)
smiles_davis, convmol_davis = smiles_graph(LOCAL_DAVIS_PATH)

In [ ]:
# This cell is from mini project
# for converting protein sequence to categorical format/numerical format
seq_voc = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
seq_dict = {v:i for i,v in enumerate(seq_voc)}
seq_dict_len = len(seq_dict)
max_seq_len = 1000   # Note that all protein data will have the same length 1000 

def seq_to_cat(prot):  # prot: protein
    x = np.zeros(max_seq_len)
    for i, ch in enumerate(prot[:max_seq_len]): 
        x[i] = seq_dict[ch]
    return x  


Read in Kiba data

In [36]:
# fpath = G_PATH + '/data_Drug_target_binding_affinity/data/kiba/'
fpath = LOCAL_KIBA_PATH 

# Read in drugs and proteins
drugs_ = json.load(open(fpath + "ligands_can.txt"), object_pairs_hook=OrderedDict)
drugs = np.array([Chem.MolToSmiles(Chem.MolFromSmiles(d),isomericSmiles=True) for d in drugs_.values()])
proteins_ = json.load(open(fpath + "proteins.txt"), object_pairs_hook=OrderedDict)
proteins = np.array(list(proteins_.values()))

# Read in affinity data
affinity = np.array(pickle.load(open(fpath + "Y","rb"), encoding='latin1'))

# Read in train/test fold  
train_fold = json.load(open(fpath + "folds/train_fold_setting1.txt"))
train_fold = [ee for e in train_fold for ee in e ]    
'''
Here all validation folds are aggregated into training set. 
If you want to train models with different architectures and/or 
optimize for model hyperparameters, we encourage you to use 5-fold 
cross validation as provided here.
'''
test_fold = json.load(open(fpath + "folds/test_fold_setting1.txt"))

# Prepare train/test data with fold indices
rows, cols = np.where(np.isnan(affinity)==False) 
drugs_tr = drugs[rows[train_fold]]    # (98545,)
proteins_tr = np.array([seq_to_cat(p) for p in proteins[cols[train_fold]]])   # (98545, 1000)
affinity_tr = affinity[rows[train_fold], cols[train_fold]]  # (98545,)

drugs_ts = drugs[rows[test_fold]] # (19709,)
proteins_ts = np.array([seq_to_cat(p) for p in proteins[cols[test_fold]]]) # (19709, 1000)
affinity_ts = affinity[rows[test_fold], cols[test_fold]]    # (19709,)

In [ ]:
# Convert to ECFP fingerprint
smileToMol = lambda x: MolFromSmiles(x)  # molecules from smiles
featurizer = dc.feat.CircularFingerprint(size=1024)

drugs_mol_tr = list(map(smileToMol, drugs_tr))
drugs_ecfp_tr = featurizer.featurize(drugs_mol_tr)
drugs_mol_ts = list(map(smileToMol, drugs_ts))
drugs_ecfp_ts = featurizer.featurize(drugs_mol_ts)

print(drugs_ecfp_tr.shape)
print(drugs_ecfp_ts.shape)

# Save to local computer
# np.savetxt(LOCAL_PATH + '/saved_data_local/drugs_ecfp_tr.csv', drugs_ecfp_tr, delimiter= ',')
# np.savetxt(LOCAL_PATH + '/saved_data_local/drugs_ecfp_ts.csv', drugs_ecfp_ts, delimiter= ',')

In [38]:
# Read drugs_edfp from the saved files
drugs_ecfp_ts = np.loadtxt('../saved_data_local/drugs_ecfp_ts.csv', delimiter = ',')
drugs_ecfp_tr = np.loadtxt('../saved_data_local/drugs_ecfp_tr.csv', delimiter = ',')

In [39]:
tr_size, drug_size = drugs_ecfp_tr.shape[0], drugs_ecfp_tr.shape[1]
ts_size = drugs_ecfp_ts.shape[0]

protein_size = max_seq_len

In [40]:
num_train, num_drugs = drugs_ecfp_tr.shape
num_prot = proteins_tr.shape[1]
drugs_tr_reshape = drugs_ecfp_tr.reshape((num_train, num_drugs, 1))
proteins_tr_reshape = proteins_tr.reshape((num_train, num_prot, 1))

# Testing data
drug_ts_reshape = drugs_ecfp_ts.reshape((drugs_ecfp_ts.shape[0], drugs_ecfp_ts.shape[1], 1))
proteins_ts_reshape = proteins_ts.reshape((proteins_ts.shape[0], proteins_ts.shape[1], 1))

In [42]:
drugs_ecfp_tr.shape

(98545, 1024)

In [44]:
# CNN for protein
cnn_input = Input(shape=(num_prot,1))
cnn_layer = Conv1D(16, 3, activation='relu', input_shape=(num_prot, 1))(cnn_input)
cnn_layer = MaxPooling1D(3)(cnn_layer)
cnn_layer = Flatten()(cnn_layer)
cnn_layer = Dropout(0.1)(cnn_layer)
cnn_layer = Dense(16, activation = 'linear')(cnn_layer)

# xs1, xs2 = nodes_ts_davis.shape[1], nodes_ts_davis.shape[2]
# as1, as2 = adj_tr_davis.shape[1], adj_tr_davis.shape[2]
X_in = Input(shape=(num_drugs, 1))
A_in = Input(shape = (num_drugs, 1))

# GNN for drugs
graph_conv = ChebConv(64, activation='relu',name='graph_input', kernel_regularizer='l1')([X_in, A_in])
graph_conv = Dropout(0.1)(graph_conv)

graph_conv = ChebConv(32, activation='relu', kernel_regularizer='l1')([graph_conv,A_in])

graph_conv = Dropout(0.1)(graph_conv)

# graph_conv = GATConv(64, activation='relu')([graph_conv,A_in])
# graph_conv = Dropout(0.5)(graph_conv)
graph_conv = ChebConv(32, activation='relu')([graph_conv, A_in])

graph_conv = Flatten()(graph_conv)
graph_conv = Dense(16, activation= 'linear')(graph_conv)

dta_concat = concatenate([graph_conv, cnn_layer])

dta_concat = Dense(512, activation='relu')(dta_concat)
dta_concat = Dense(16, activation='relu')(dta_concat)
dta_concat = Dense(1, activation='softmax')(dta_concat)

model = Model(inputs={'graph_input':[X_in, A_in], 'cnn_input':cnn_input}, outputs=dta_concat)



model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           [(None, 1024, 1)]    0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           [(None, 1024, 1)]    0                                            
__________________________________________________________________________________________________
graph_input (ChebConv)          (None, 1024, 64)     128         input_26[0][0]                   
                                                                 input_27[0][0]                   
__________________________________________________________________________________________________
dropout_23 (Dropout)            (None, 1024, 64)     0           graph_input[0][0]          

In [45]:
history = model.fit({'graph_input': [drugs_tr_reshape, drugs_tr_reshape], 'cnn_input': proteins_tr_reshape }, affinity_tr,batch_size=28, epochs=50,steps_per_epoch=10)

Epoch 1/50
10/10 [==============================] - 2s 63ms/step - loss: 117.0641 - mean_squared_error: 114.4603
Epoch 2/50
10/10 [==============================] - 0s 43ms/step - loss: 116.1204 - mean_squared_error: 113.7198
Epoch 3/50
10/10 [==============================] - 0s 47ms/step - loss: 116.6083 - mean_squared_error: 114.4017
Epoch 4/50
10/10 [==============================] - 1s 59ms/step - loss: 116.2542 - mean_squared_error: 114.2350
Epoch 5/50
10/10 [==============================] - 1s 52ms/step - loss: 117.3981 - mean_squared_error: 115.5598
Epoch 6/50
10/10 [==============================] - 1s 52ms/step - loss: 115.9933 - mean_squared_error: 114.3276
Epoch 7/50
10/10 [==============================] - 1s 50ms/step - loss: 118.5910 - mean_squared_error: 117.0901
Epoch 8/50
10/10 [==============================] - 0s 44ms/step - loss: 117.5895 - mean_squared_error: 116.2445
Epoch 9/50
10/10 [==============================] - 1s 54ms/step - loss: 116.0487 - mean_squared

In [46]:
drug_ts_reshape = drugs_ecfp_ts.reshape((drugs_ecfp_ts.shape[0], drugs_ecfp_ts.shape[1], 1))
proteins_ts_reshape = proteins_ts.reshape((proteins_ts.shape[0], proteins_ts.shape[1], 1))
print("Evaluate on test data")
results = model.evaluate({'graph_input': [drug_ts_reshape, drug_ts_reshape], 'cnn_input': proteins_ts_reshape }, affinity_ts, batch_size=128)
print("test MSE loss is:", results)

Evaluate on test data
154/154 [==============================] - 4s 22ms/step - loss: 115.5841 - mean_squared_error: 115.5814
test MSE loss is: [115.58407592773438, 115.5813980102539]
